In [3]:
import requests
import json

In [4]:
with open('../config.json', 'r') as jsonfile:
    data = json.load(jsonfile)
api_key = data['api_key']

In [27]:
url = f'https://api.nasa.gov/insight_weather/?api_key={api_key}&feedtype=json'

In [28]:
def get_mars_weather():
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        print(data)

        sol_keys = data['sol_keys']
        print(f"Latest available Sols: {sol_keys}")

        for sol in sol_keys:
            sol_data = data[sol]
            print(f"\nWeather for Sol {sol}:")
            if 'AT' in sol_data:
                print(f"Average Temperature: {sol_data['AT']['av']}°C")
            else:
                print("Temperature data is not available.")
            
            if 'HWS' in sol_data:
                print(f"Average Wind Speed: {sol_data['HWS']['av']} m/s")
            else:
                print("Wind speed data is not available.")
            
            if 'PRE' in sol_data:
                print(f"Average Atmospheric Pressure: {sol_data['PRE']['av']} Pa")
            else:
                print("Pressure data is not available.")
            
            if 'WD' in sol_data and sol_data['WD'].get('most_common'):
                print(f"Most Common Wind Direction: {sol_data['WD']['most_common']['compass_point']}")
            else:
                print("Wind direction data is not available.")

            print(f"Mars Season: {sol_data['Season']}")
            print(f"First Data UTC: {sol_data['First_UTC']}")
            print(f"Last Data UTC: {sol_data['Last_UTC']}")

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except KeyError as e:
        print(f"Error processing data: {e} - The structure of the data might have changed or it's currently unavailable.")

In [29]:
get_mars_weather()

{'sol_keys': [], 'validity_checks': {'1219': {'AT': {'sol_hours_with_data': [6, 7, 8, 9, 10, 11, 12], 'valid': False}, 'HWS': {'sol_hours_with_data': [6, 7, 8, 9, 10, 11, 12], 'valid': False}, 'PRE': {'sol_hours_with_data': [6, 7, 8, 9, 10, 11, 12], 'valid': False}, 'WD': {'sol_hours_with_data': [6, 7, 8, 9, 10, 11, 12], 'valid': False}}, 'sol_hours_required': 18, 'sols_checked': ['1219']}}
Latest available Sols: []


## Solar flare

In [22]:
# we can change the dates 
solar_flare_url = f'https://api.nasa.gov/DONKI/FLR?startDate=2024-01-01&endDate=2024-02-01&api_key={api_key}'

how it looks like

```python
[{"flrID":"2016-01-01T23:00:00-FLR-001","instruments":[{"displayName":"GOES15: SEM/XRS 1.0-8.0"}],"beginTime":"2016-01-01T23:00Z","peakTime":"2015-01-02T00:10Z","endTime":null,"classType":"M2.3","sourceLocation":"S21W73","activeRegionNum":12473,"linkedEvents":[{"activityID":"2016-01-01T23:12:00-CME-001"},{"activityID":"2016-01-02T02:48:00-SEP-001"},{"activityID":"2016-01-02T04:30:00-SEP-001"}],"link":"https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/FLR/9963/-1"},{"flrID":"2016-01-28T11:48:00-FLR-001","instruments":[{"displayName":"GOES15: SEM/XRS 1.0-8.0"}],"beginTime":"2016-01-28T11:48Z","peakTime":"2016-01-28T12:02Z","endTime":"2016-01-28T12:56Z","classType":"C9.6","sourceLocation":"N03W47","activeRegionNum":12488,"linkedEvents":[{"activityID":"2016-01-28T12:24:00-CME-001"}],"link":"https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/FLR/10122/-1"}]
```

In [25]:
try:
    response = requests.get(solar_flare_url)
    response.raise_for_status()
    data = response.json()
    print(len(data))
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

50
